# 03. CODE REPRESENTATION

1. [x] введение
2. [x] AST
3. [x] code2vec
4. [x] упражнение
5. [x] ссылки

# 1. Введение

*Представление кода* --- это преобразование кода в некоторый математический объект. Какие выдвигаются требования к  этому преобразованию?
- важная информация сохраняется
- ненужная информация не сохраняется
- с результирующим объектом работать проще, чем и с исходным кодом

"Важность" и "ненужность" --- зависит от тех задач, которые предполагается решать. В английском языке используется два термина "representation" и "embedding". В последнем случае, в качестве математического объекта рассматривается точка метрического пространства (т.е. множество с метрикой).

Выбор представления является очень важным шагом, поскольку на этом этапе почти всегда происходит потеря информации. Следовательно,
- если теряется важная информация, то это может отрицательно сказаться на качестве решения задачи (вплоть до того, что в задаче классификации классы могут перестать разделяться)
- если теряется ненужная информация, то это может пойти задаче на пользу, поскольку уменьшается количество степеней свободы

Чаще всего код представляется в виде векторов с вещественными координатами фиксированной размерности.

# 2. AST

*Abstract syntax tree (AST)* --- абстрактное синтаксическое дерево.

В информатике AST --- это конечное помеченное ориентированное дерево, в котором внутренние вершины соответствуют операторам языка программирования, а листья --- операндам (переменные и константы).

#### Пример

> Алгоритм Евклида и абстрактное синтаксическое дерево:
>
> [`while b ≠ 0
    if a > b
        a := a − b
    else
        b := b − a
return a`](#code)
>
> ![AST](https://upload.wikimedia.org/wikipedia/commons/thumb/c/c7/Abstract_syntax_tree_for_Euclidean_algorithm.svg/425px-Abstract_syntax_tree_for_Euclidean_algorithm.svg.png)

#### Пакет ast

https://docs.python.org/3/library/ast.html

![ast](./res/03_ast.png)

In [1]:
import ast
import astpretty
from pprint import pprint

In [2]:
source = '''\
def f(a, b):
    while b != 0:
        if a > b:
            a = a - b
        else:
            b = b - a
    return a'''
tree = ast.parse(source)

In [3]:
astpretty.pprint(tree, show_offsets=False)

Module(
    body=[
        FunctionDef(
            name='f',
            args=arguments(
                args=[
                    arg(arg='a', annotation=None),
                    arg(arg='b', annotation=None),
                ],
                vararg=None,
                kwonlyargs=[],
                kw_defaults=[],
                kwarg=None,
                defaults=[],
            ),
            body=[
                While(
                    test=Compare(
                        left=Name(id='b', ctx=Load()),
                        ops=[NotEq()],
                        comparators=[Num(n=0)],
                    ),
                    body=[
                        If(
                            test=Compare(
                                left=Name(id='a', ctx=Load()),
                                ops=[Gt()],
                                comparators=[Name(id='b', ctx=Load())],
                            ),
                            body=[
                   

#### Библиотека tree-sitter

https://tree-sitter.github.io/tree-sitter/

![tree-sitter](./res/03_tree-sitter.png)

# 3. code2vec

## Цель

Хотим представить фрагмент кода вектором фиксированной длины. Этот вектор далее хотим использовать для предсказания семантических свойств исходного фрагмента кода.
 

## Идея

Решим задачу предсказания имени метода по его исходному коду. В результате обучения модели получим вектора, которые являются хорошим представлением кода.

## Статья

![](./res/03_code2vec_paper.png)

## Модель

### Обозначения

Пусть $C$ --- фрагмент кода. Тогда AST --- это шестёрка $⟨N, T, X, s, δ, φ⟩$, где
- $N$ --- множество нетерминальных вершин,
- $T$ --- множество терминальных вершин,
- $X$ --- множество значений,
- $s ∈ N$ --- корневая вершина (корень),
- $δ: N → (N \cup T )^*$ --- функция, которая отображает нетерминальную вершину в список её потомков,
- $φ: T → X$ --- функция, которая отображает терминальную вершину в связанную с ней значение.

Каждая вершина за исключением корня является потомком ровной одной вершины (отсутствие циклов).

*AST-путь* длины $k$ --- это последовательность вида $n_1 d_1 ... n_k d_k n_{k+1}$, где
- $n_1$, $n_{k+1} ∈ T$ --- терминальные вершины, а
- $n_i ∈ N$ ($i ∈ [2..k]$) --- нетерминальные и
- $d_i ∈ \{↑, ↓\}$ ($i ∈ [1..k]$) --- направление движения в AST (вверх или вниз).
  - если $d_i = ↑$, то $n_i ∈ δ(n_{i+1})$
  - если $d_i = ↓$, то $n_{i+1} ∈ δ(n_i)$

Для AST-пути $p$ положим $start(p) := n_1$ и $end(p) := n_{k+1}$.

Пусть $p$ --- AST-путь, тогда *контекстный путь* --- это тройка $⟨x_s, p, x_t⟩$, где
- $x_s = φ(start(p))$
- $x_t = φ(end(p))$

т.е. значения, связанные с начальной и конечной вершинами пути $p$ (или контекст пути).

#### Пример

> Для выражения `x = 7;` имеем такой пример контекстного пути
> $$⟨x, (NameExpr ↑ AssignExpr ↓ IntegerLiteralExpr), 7 ⟩.$$

Пусть $C$ --- фрагмент кода и $⟨N, T, X, s, δ, φ⟩$ --- соответствующее AST-дерево. Через $TPairs$ обозначим множество всех пар различных терминальных вершин, то есть
$$ TPairs(C) = \{(term_i , term_j) | term_i, term_j ∈ termNodes(C) ∧ i \neq j\},$$
где $termNodes$ --- множество всех терминальных вершин для фрагмента кода $C$.

Положим
$$ Rep(C) = \{ (x_s , p, x_t )| ∃(term_s, term_t) ∈ TPairs(C): x_s = φ(term_s) ∧ x_t = φ(term_t) ∧ start(p) = term_s ∧ end(p) = term_t\}.$$
Таким образом, по фрагменту кода $C$ определяется множество всех контекстных путей вида $⟨x_s, p, x_t⟩$,
 где $p$ --- путь в соответствующем AST.
 
#### Пример

> `
boolean f(Object target) {
    for (Object elem: this.elements){
        if (elem.equals(target)) {
            return true;
        }
    }
}
`
>
> ![ast](./res/03_code2vec_ast.png)
> Эллипсы --- терминальные и нетерминальные вершины, прямоугольники --- значения.
>
> Контекстный путь 1:
>$$(elements, Name ↑ FieldAccess ↑ Foreach ↓ Block ↓ IfStmt ↓ Block ↓ Return ↓ BooleanExpr, true).$$

## Модель

Модель имеет следующую архитектуру

![overview](./res/03_code2vec_overview.png)

## Контекстный вектор

Модель использует два (обучаемых) словаря для построения эмбеддингов:
- $path\_vocab$ и
- $value\_vocab$.

Словари реализуются матрицами, в которых каждой строке соответствует путь ($path\_vocab$) или значение ($value\_vocab$). Имеем
$$path\_vocab  \in R^{|P| \times d},$$
$$value\_vocab \in R^{|X| \times d},$$

где $X$ --- множество значений для терминальных вершин AST-дерева, которые мы имеем на этапе обучения,
а $P$ --- множество всех AST-путей.
Чтобы ограничить размер данных для обучения и уменьшить разреженность данные, можно ограничить различные параметры путей (например, максимальная длина пути $k$). Эти значения определяются эмпирически как гиперпараметры модели.

Эмбеддинги получаются выбором соответствующей строки матрицы.

Например, $value\_vocab$ содержит строки для каждого из значений `boolean`, `target` и `Object`.
А $path\_vocab$ содержит строки, который соответствуют каждому из AST-путей (без учёта значений), например, крвсный путь 1 это:
$$ Name ↑ FieldAccess ↑ Foreach ↓ Block ↓ IfStmt ↓ Block ↓ Return ↓ BooleanExpr.$$

Значения этих матриц инициализируются случайным образом. Матрицы обучаются, одновременно с сетью.

Размерность $d$ определяется эмпирически, ограничено временем обучения, сложностью модели и GPU-памятью и обычно находится в диапазоне от 100 до 500. Для удобства, эмбеддинги путей и значений будут иметь одинаковый размер $d$, но в общем случае могу отличаться. 

Через $B = \{b_1 , ..., b_n\}$ обозначим множество контекстных путей, которые были извлечены из фрагмента кода.
Множество $B$ используется для обучения модели.

Пусть $b_i = ⟨x_s, p_j, x_t⟩$ --- один из контекстных путей, при этом ${x_s, x_t} \in X$ --- значения терминальных вершин и $p_j \in P$ --- соединяющий их путь. Каждый компонент контекстного пути через матрицы превращается в соответствующий эмбеддинг. Три этих эмбеддинга каждого контекстного пути конкатенируются в единый *контекстный вектор (context vector)*: $c_i ∈ R^{3d}$. Этот контекстный вектор является представлением контекстного пути:
$$c_i = embedding(⟨x_s, p_j, x_t⟩) = [value\_vocab_s; path\_vocab_j; value\_vocab_t] ∈ R^{3d}.$$

### Полносвязный вектор

Поскольку каждый контекстный вектор $c_i$ получается конкатенацией трёх независимых векторов, было бы неплохо их "перемешать". Это делает полносвязный слой.

"Перемешивание" осуществляется раздельно для каждого контекстного вектора, но используется один и тот же слой. 

Через $c̃_i$ обозначается выход полносвязного слоя:
$$c̃_i = tanh(W c_i),$$
где $W \in R^{d \times 3d}$ --- обучаемая матрица весов.

Высота матрицы весов $W$ определяет размерность вектора $c̃_i$ и для удобства имеет тот же размер ($d$), как и раньше. В общем случае высота $W$ может быть разной; это повлияет на размер конечного *кодового вектора (code vector)*. Таким образом полносвязный слой сжимает контекстный вектор размера $3d$ в комбинированный контекстный вектор размера $d$ (combined context vectors), умножая его на матрицу весов, а затем применяет функцию $tanh$ к каждому элементу вектора отдельно.

### Attention

Attention-механизм вычисляет средневзвешенное значение по комбинированным контекстным векторам (combined context vectors), и его основная задача состоит в вычислении скалярного веса для каждого из них. Attention-вектор $a \in R^d$ инициализируется случайным образом и обучается одновременно с сетью. Для заданных комбинированных контекстных векторов (combined context vectors) $\{c̃_1, ..., c̃_n\}$, вес $α_i$ каждого $c̃_i$ вычисляется как нормализованное скалярное произведение между комбинированным контекстным вектором (combined context vectors) и глобальным attention-вектором $a $:
$$attention-вес~ α_i = \frac{exp(c̃_i^T a)}{\sum_{j=1}^{n}exp(c̃_j^T a)}.$$

Экспоненты используются только для того, чтобы сделать attention-веса положительными, и они делятся на суммы, чтобы получить значение было от $0$ до $1$ (как в случае softmax).
Агрегированный кодовый вектор (code vector) $v \in R^d$, представляющий весь фрагмент кода, представляет собой линейную комбинацию комбинированных контекстных векторов (combined context vectors) $\{c̃_1, ..., c̃_n\}$ с attention-весами:
$$кодовый~ вектор~ (code~ vector)~ v = \sum_{i=1}^n α_i c̃_i.$$

Таким образом, attention-веса неотрицательны, их сумма равна $1$ и они используются как коэффициенты комбинированных контекстных векторов (combined context vectors) $c̃_i$. Поэтому attention-механизм можно рассматривать как взвешенное усреднение, при котором веса обучаются и рассчитываются по отношению к другим элементам из множества контекстных путей (path-contexts).

### Обучение

Прогноз целевой метки (tag) выполняется с использованием кодового вектора (code vector). Для этого необходим словать целевых меток (tag), который обучается одновременно с сетью:
$$tags\_vocab ∈ R^{|Y| \times d},$$
где $Y$ --- набор значений целевых меток (tag), собранных из обучающей выборки. Как и раньше, эмбеддинг $tag_i$ --- это $i$-ая строка таблицы $tags\_vocab$. Например, $tags\_vocab$ содержит строки для каждой из целевых меток `contains`, `matches` и `canHandle`. Итоговое распределение модели $q(y)$ вычисляется как скалярное произведение (нормализованное по softmax) между кодовым вектором (code vector) $v$ и каждым из эмбеддингов целевых метрик:
$$for~ y_i \in Y: q(y_i) = \frac{exp(v^T tags\_vocab_i)}{\sum_{j} exp(v^T tags\_vocab_j)}.$$


В целом, следующие компоненты модели проходят обучение:
- эмбеддинги для путей и значений (матрицы $path\_vocab$ и $value\_vocab$)
- полносвязный слой (матрица $W$)
- attention-вектор ($a$)
- эмбеддинги для целевых меток ($tags\_vocab$).


При обучении в качестве функции потерь выступает кросс-энтропия между предсказанным распределением $q$ и истинным распределением $p$ (значение $1$ фактической метке в обучающем примере и $0$ в противном случае).

Сеть обучается с использованием любого алгоритма, основанного на градиентном спуске, и стандартного подхода обратного распространения ошибки обучения по каждому из обучаемых параметров.

## Реализация

- https://code2vec.org/
- https://github.com/bentrevett/code2vec
- https://github.com/d6ms/pytorch-code2vec

# 4. Упражение

Используя готовую реализацию code2vec, исследовать свойства такого представления кода.
1. Буду ли близким вектора, соответствующие семантически близким фрагментам кода?
2. Какую меру близости лучше всего использовать?

# 5. Полезные ссылки

- [AST](https://en.wikipedia.org/wiki/Abstract_syntax_tree)
- Alon et al --- Code2Vec: Learning distributed representations of code, 2019
  * [paper](https://arxiv.org/abs/1803.09473)
  * [DOI](https://doi.org/10.1145/3290353)
  * [presentation @ FLOC'18](https://www.youtube.com/watch?v=ya74zYDhSnM)
  * [presentation @ POPL'19](https://www.youtube.com/watch?v=EJ8okcxL2Iw)
  * [paper reading](https://www.youtube.com/watch?v=4j8mif-ybyw)
  * [demo](https://code2vec.org/)
  * [source code](https://github.com/tech-srl/code2vec)
  * [more code](https://github.com/sonoisa/code2vec)